In [1]:
from tensorrtserver.api import grpc_service_pb2
from functools import partial
import grpc_gcp_caip_pb2
import grpc_image_client
import requests
import argparse


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('-v', '--verbose', action="store_true", required=False, default=False)
parser.add_argument('-a', '--async', dest="async_set", action="store_true", required=False)
parser.add_argument('--streaming', action="store_true", required=False, default=False)
parser.add_argument('-m', '--model-name', type=str, required=True)
parser.add_argument('-x', '--model-version', type=int, required=False)
parser.add_argument('-b', '--batch-size', type=int, required=False, default=1)
parser.add_argument('-c', '--classes', type=int, required=False, default=1)
parser.add_argument('-s', '--scaling', type=str, choices=['NONE', 'INCEPTION', 'VGG'],required=False, default='NONE')
parser.add_argument('-u', '--url', type=str, required=False, default='localhost:8001')
#parser.add_argument('-i','--image_filename', type=str, default=None)
parser.add_argument('image_filename', type=str, nargs='?', default=None)

FLAGS = parser.parse_args(args=['-m','resnet50_netdef',
#                                '-u','35.225.226.0:5000',
#                                '-u','35.202.30.17:5000',
                                '-u','127.0.0.1:5000',
                                '-s','INCEPTION',
                                '/workspace/images/mug.jpg'])


In [3]:
import googleapiclient.discovery

In [17]:
VERSION_NAME='v010'
MODEL_NAME='triton1'
PROJECT_NAME='tsaikevin-inference'
ENDPOINT='https://alpha-ml.googleapis.com/v1'

In [18]:
predict_url = "{}/projects/{}/models/{}/versions/{}:predict".format(ENDPOINT, PROJECT_NAME, MODEL_NAME, VERSION_NAME)
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Bearer `gcloud auth print-access-token`'
}
predict_url

'https://alpha-ml.googleapis.com/v1/projects/tsaikevin-inference/models/triton1/versions/v010:predict'

In [19]:
# Prepare request for Status gRPC
status_request = grpc_service_pb2.StatusRequest(model_name=FLAGS.model_name)
caip_request = grpc_gcp_caip_pb2.CaipRequest(request_type=grpc_gcp_caip_pb2.TYPE_STATUS_REQUEST, status_request = status_request)

# CaipRequest send, StatusResponse receive
response = requests.request("POST", predict_url, headers=headers, data = caip_request.SerializeToString()).content

response
#status_response = grpc_service_pb2.StatusResponse().FromString(response)

b'{\n  "error": {\n    "code": 401,\n    "message": "Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",\n    "status": "UNAUTHENTICATED"\n  }\n}\n'

In [7]:
with open('caip_status.msg','wb') as caip_status:
    caip_status.write(caip_request.SerializePartialToString())

In [8]:
service = googleapiclient.discovery.build('ml','v1')

/usr/local/lib/python3.6/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [23]:
# Make sure the model matches our requirements, and get some
# properties of the model that we need for preprocessing
input_name, output_name, c, h, w, format, dtype = grpc_image_client.parse_model(status_response, FLAGS.model_name, FLAGS.batch_size, FLAGS.verbose)

In [24]:
filledRequestGenerator = partial(grpc_image_client.requestGenerator, input_name, output_name, c, h, w, format, dtype, FLAGS)

In [25]:
# Send requests of FLAGS.batch_size images. If the number of
# images isn't an exact multiple of FLAGS.batch_size then just
# start over with the first images until the batch is filled.
result_filenames = []
infer_requests = []
infer_responses = []

In [26]:
for infer_request in filledRequestGenerator(result_filenames):
    caip_request = grpc_gcp_caip_pb2.CaipRequest(request_type=grpc_gcp_caip_pb2.TYPE_INFER_REQUEST, infer_request = infer_request)
    infer_responses.append(grpc_service_pb2.InferResponse().FromString(requests.request("POST", predict_url, headers=headers, data = caip_request.SerializeToString()).content))
    
idx = 0
for infer_response in infer_responses:
    print("Request {}, batch size {}".format(idx, FLAGS.batch_size))
    grpc_image_client.postprocess(infer_response.meta_data.output, result_filenames[idx], FLAGS.batch_size)
    idx += 1

Request 0, batch size 1
Image '/workspace/images/mug.jpg':
    504 (COFFEE MUG) = 0.7773650288581848


In [7]:
service = googleapiclient.discovery.build('ml','v1')

/usr/local/lib/python3.6/dist-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [11]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/workspace/src/clients/python/api_v1/examples/client_secret_291406910847-af8ojm05iri31hg5elr8rg4fk5u2pse8.apps.googleusercontent.com.json"

In [21]:
!curl -X GET -k -H "Content-Type: application/json" \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "https://alpha-ml.googleapis.com/v1/projects/tsaikevin-inference/models/triton1/versions/v010"

{
  "name": "projects/tsaikevin-inference/models/triton1/versions/v010",
  "deploymentUri": "gs://tsaikevin-inference-model-store/model_repository",
  "createTime": "2020-05-21T06:47:54Z",
  "lastUseTime": "2020-05-21T07:34:00Z",
  "state": "READY",
  "etag": "oN3nAn4F/Sg=",
  "machineType": "n1-standard-4",
  "acceleratorConfig": {
    "count": "1",
    "type": "NVIDIA_TESLA_T4"
  },
  "container": {
    "image": "gcr.io/tsaikevin-inference/caip-triton:latest",
    "args": [
      "trtserver",
      "--model-repository=$(AIP_STORAGE_URI)"
    ],
    "ports": [
      {
        "containerPort": 5000
      }
    ]
  },
  "routes": {
    "predict": "/v1/models/triton1/versions/v010:predict",
    "health": "/v1/models/triton1/versions/v010"
  }
}


In [14]:
!curl -X POST -k -H "Content-Type: application/json" \
    -d caip_status.msg \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "https://alpha-ml.googleapis.com/v1/projects/tsaikevin-inference/models/triton1/versions/v010:predict"

{
  "error": {
    "code": 500,
    "message": "\u003c!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.01 Transitional//EN\"\n  \"http://www.w3.org/TR/html4/loose.dtd\"\u003e\n\u003chtml\u003e\n  \u003chead\u003e\n    \u003ctitle\u003egoogle.protobuf.message.DecodeError: Error parsing message // Werkzeug Debugger\u003c/title\u003e\n    \u003clink rel=\"stylesheet\" href=\"?__debugger__=yes&amp;cmd=resource&amp;f=style.css\"\n        type=\"text/css\"\u003e\n    \u003c!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. --\u003e\n    \u003clink rel=\"shortcut icon\"\n        href=\"?__debugger__=yes&amp;cmd=resource&amp;f=console.png\"\u003e\n    \u003cscript src=\"?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js\"\u003e\u003c/script\u003e\n    \u003cscript src=\"?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js\"\u003e\u003c/script\u003e\n    \u003cscript type=

In [11]:
tracer_text = "ls -l"
caip_tracer = grpc_gcp_caip_pb2.CaipRequest(request_type=grpc_gcp_caip_pb2.TYPE_TRACER, trace_message = tracer_text)
response = requests.request("POST", predict_url, headers=headers, data = caip_tracer.SerializeToString()).content
response

b'{\n  "error": {\n    "code": 401,\n    "message": "Request had invalid authentication credentials. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",\n    "status": "UNAUTHENTICATED"\n  }\n}\n'

In [12]:
with open('caip_trace.msg','wb') as caip_trace:
    caip_trace.write(caip_tracer.SerializePartialToString())

In [16]:
caip_tracer

request_type: TYPE_TRACER
trace_message: "ls -l"

In [20]:
!curl -X POST -k -H "Content-Type: application/json" \
    -d caip_trace.msg \
    -H "Authorization: Bearer `gcloud auth print-access-token`" \
    "https://alpha-ml.googleapis.com/v1/projects/tsaikevin-inference/models/triton1/versions/v010:predict"

{
  "error": {
    "code": 500,
    "message": "\u003c!DOCTYPE HTML PUBLIC \"-//W3C//DTD HTML 4.01 Transitional//EN\"\n  \"http://www.w3.org/TR/html4/loose.dtd\"\u003e\n\u003chtml\u003e\n  \u003chead\u003e\n    \u003ctitle\u003egoogle.protobuf.message.DecodeError: Error parsing message // Werkzeug Debugger\u003c/title\u003e\n    \u003clink rel=\"stylesheet\" href=\"?__debugger__=yes&amp;cmd=resource&amp;f=style.css\"\n        type=\"text/css\"\u003e\n    \u003c!-- We need to make sure this has a favicon so that the debugger does\n         not by accident trigger a request to /favicon.ico which might\n         change the application state. --\u003e\n    \u003clink rel=\"shortcut icon\"\n        href=\"?__debugger__=yes&amp;cmd=resource&amp;f=console.png\"\u003e\n    \u003cscript src=\"?__debugger__=yes&amp;cmd=resource&amp;f=jquery.js\"\u003e\u003c/script\u003e\n    \u003cscript src=\"?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js\"\u003e\u003c/script\u003e\n    \u003cscript type=